In [13]:
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from selenium.webdriver.common.by import By
from itertools import count
import time
from tqdm import tqdm_notebook
import pdb
import re
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [14]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
job='데이터' #검색 keyword
company_name = []
position_name = []
requirements =[]
preferences = []

In [15]:
driver.get(f'https://www.wanted.co.kr/search?query={job}')

# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 끝까지 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 대기
    time.sleep(1)

    # 스크롤 내린 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [18]:
address=[]
i=1
while True:
    try:
        elem = driver.find_element(By.XPATH, f'//*[@id="__next"]/div[4]/div/div[2]/div[3]/div/div/div[2]/ul/li[{i}]/div/a').get_attribute('href')
        address.append(elem)
        i+=1
    except Exception as e:
        break

In [21]:
for job_address in address:
    driver.get(job_address)
    time.sleep(3)
    try:
        position=driver.find_element(By.XPATH,'//*[@id="__next"]/div[3]/div[1]/div[1]/div[1]/section[2]/h2').text
        company=driver.find_element(By.XPATH,'//*[@id="__next"]/div[3]/div[1]/div[1]/div[1]/section[2]/div[1]/h6/a').text
        requirement=driver.find_element(By.XPATH,'//*[@id="__next"]/div[3]/div[1]/div[1]/div[1]/div[2]/section[1]/p[3]/span').text
        prefer=driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/div[1]/div[1]/div[2]/section[1]/p[4]/span').text
        #location=driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div[1]/div[1]/div[1]/section[2]/div[2]').text
    except Exception as e:
        pass
        
    company_name.append(company)
    position_name.append(position)
    requirements.append(requirement)
    preferences.append(prefer)
  

In [23]:
job_df = pd.DataFrame({'회사명' : company_name, '직무' : position_name, '링크' : address, '자격요건' : requirements, '우대사항' : preferences})
job_df.to_csv("job.csv", index = False)

In [24]:
job_df

,회사명,직무,링크,자격요건,우대사항
0,데이터라이즈,Backend Engineer,https://www.wanted.co.kr/wd/81841,• 3년 이상의 실무 경험 또는 그에 준하는 이해도가 있으신 분 \n• 1개 이상의...,• 좋은 코드와 설계를 위한 고민 그리고 테스트의 중요성을 인지하고 계신 분\n• ...
1,데이터라이즈,Data Engineer,https://www.wanted.co.kr/wd/81834,"• 데이터 수집, 전처리 중 1개 이상 구축 경험이 있는 1년 이상의 경력자 또는 ...",• 좋은 코드와 설계를 위한 고민 그리고 테스트의 중요성을 인지하고 계신 분\n• ...
2,데이터비,AI Engineer (ML/DL),https://www.wanted.co.kr/wd/64852,"- Python을 이용한 개발 경험 (pytorch, tensorflow, scik...",- 데이터와 소프트웨어 기술이 사람간의 의사소통을 더 원활하게 도울 수 있다고 믿는...
3,윙잇,[윙잇] 데이터 분석가,https://www.wanted.co.kr/wd/133324,■[이런 분과 함께하고 싶어요]\n• 데이터 분석 관련 경력 3년 이상 보유하신 분...,■[이런 분이면 더욱 좋아요]\n• 프로그래밍 언어 구현 능력 있으신 분\n• 컴퓨...
4,로이드케이,빅데이터 솔루션 데이터 엔지니어,https://www.wanted.co.kr/wd/144903,"ㆍ서버 (Linux, Windows Server) 개념 이해 및 경험\nㆍDBMS ...","ㆍ컴퓨터/정보통신/보안/시스템 공학계열 전공자\nㆍ정보처리(산업)기사, 리눅스마스터..."
...,...,...,...,...,...
368,렉스소프트,통계 데이터 전문가,https://www.wanted.co.kr/wd/137540,• 대학졸업자(졸업예정자도 가능) \n• R을 활용한 데이터 분석 능력,"• 통계학과 또는 통계 관련 학과 전공(복수전공, 부전공 포함)\n• 다양한 통계분..."
369,오토노머스에이투지,자율주행차 운영 및 데이터 관리(세종),https://www.wanted.co.kr/wd/135547,"• 경력: 무관(신입도 지원 가능)(연구원, 매니저급)\n• 기타 필수 사항\n ...","• 공학계열\n• 1종대형운전면허\n• 차량소지자, 인근거주자"
370,소프트포럼,데이터보안 개발자,https://www.wanted.co.kr/wd/127430,• 학력 : 대졸 이상\n• 경력 : 경력 5년 이상\n• C/C++ 개발 가능자\...,"• DBMS(Oracle, MS-SQL 등) 경험자\n• CC인증, GS인증 경험자..."
371,지오소프트,KT FMS IO SM SYSTEM ENGINEER & DBA : OS / MIDD...,https://www.wanted.co.kr/wd/83498,"• CENTOS 서버 관리 능력, LOG 분석\n• JBOSS NGINX APACH...",• 5년 이상 경력자 우대( 연봉 반영 )
